In [10]:
from sklearn.metrics import mean_absolute_error
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.preprocessing import MinMaxScaler

# Cargar las neuronas del gonza
rn_1 = joblib.load('ml/neural_network_device_id_1.joblib')
# Cargar los datos del CSV
df = pd.read_csv('csv/device_id_1_2024-11-03_2024-11-20.csv')
df['hour'] = pd.to_datetime(df['hour'])  # Asegúrate de manejar la columna 'hour'


# Calcular la columna 'result' en base a la predicción de humedad
def predict_soil_moisture(row):
    air_temperature = row['avg_air_temperature']
    air_humidity = row['avg_air_humidity']
    soil_humidity = row['avg_soil_humidity']

    new_data = np.array([[air_temperature, air_humidity, soil_humidity]])
    predicted_change = rn_1.predict(new_data)[0][0]
    return round(soil_humidity + predicted_change, 2)


# Aplicar la predicción de la humedad proyectada para cada fila
df['soil_humidity_predicted'] = df.apply(predict_soil_moisture, axis=1)

# Crear la variable objetivo 'is_optimal' (0 = No es óptimo, 1 = Es óptimo)
optimal_range_min = 40  # Ajusta según las necesidades
optimal_range_max = 58  # Ajusta según las necesidades
df['is_optimal'] = ((df['soil_humidity_predicted'] >= optimal_range_min) & (
        df['soil_humidity_predicted'] <= optimal_range_max)).astype(int)

# Seleccionar las variables predictoras y la variable objetivo
X = df[['avg_air_temperature', 'avg_air_humidity', 'soil_humidity_predicted']].values
y = df['is_optimal'].values

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Escalar los datos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo de Random Forest
clf = RandomForestClassifier(n_estimators=150, random_state=42)
clf.fit(X_train_scaled, y_train)

# Evaluar el modelo
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

# Calcular las métricas F1, Recall y Precision
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"\nF1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")

# Calcular el MAE
mae = mean_absolute_error(y_test, clf.predict(X_test))
print(f"Mean Absolute Error (MAE): {mae:.2f}")


# Realizar predicciones con nuevas entradas
new_data_for_model = np.array([[14.9, 72.2, 50.67]])  # Usa el resultado de tu modelo anterior para la humedad proyectada
new_data_scaled = scaler.transform(new_data_for_model)
is_optimal_prediction = clf.predict(new_data_scaled)[0]
print(is_optimal_prediction)
if is_optimal_prediction:
    print("La próxima hora es óptima para fertiirrigar.")
else:
    print("La próxima hora NO es óptima para fertiirrigar.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━